In [27]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,desc

In [2]:
spark=SparkSession.builder.appName('Practise').getOrCreate()

In [3]:
spark

In [4]:
print(spark.version)

3.5.2


In [14]:
data = spark.read.format('csv').\
option('inferSchema','true').\
option('header','true').\
option('path','operations_management.csv').\
load()

In [16]:
data.printSchema()

root
 |-- description: string (nullable = true)
 |-- industry: string (nullable = true)
 |-- level: integer (nullable = true)
 |-- size: string (nullable = true)
 |-- line_code: string (nullable = true)
 |-- value: integer (nullable = true)



In [17]:
df=spark.read.csv('operations_management.csv',inferSchema=True,header=True)

In [20]:
df.show()

+--------------------+--------------------+-----+---------------+---------+-----+
|         description|            industry|level|           size|line_code|value|
+--------------------+--------------------+-----+---------------+---------+-----+
|Awareness of clim...|               total|    0| 6–19 employees| C0300.01|13080|
|Awareness of clim...|               total|    0|20–49 employees| C0300.01| 3348|
|Awareness of clim...|               total|    0|50–99 employees| C0300.01| 1089|
|Awareness of clim...|               total|    0| 100+ employees| C0300.01| 1023|
|Awareness of clim...|Agriculture, fore...|    1|          total| C0300.01| 2364|
|Awareness of clim...|         Agriculture|    2|          total| C0300.01| 1683|
|Awareness of clim...|  Commercial fishing|    2|          total| C0300.01|   27|
|Awareness of clim...|  Forestry & logging|    2|          total| C0300.01|  126|
|Awareness of clim...|Agriculture, fore...|    2|          total| C0300.01|  528|
|Awareness of cl

In [31]:
df_new=df.select('industry','value').filter((col('value') > 500) & (col('industry')!= 'total')).orderBy(desc('value'))

In [32]:
df_new.show(10)

+--------------------+-----+
|            industry|value|
+--------------------+-----+
|        Construction| 6030|
|        Construction| 5904|
|        Construction| 5229|
|Accommodation & f...| 5058|
|        Construction| 4965|
|        Construction| 4959|
|Accommodation & f...| 4950|
|        Construction| 4686|
|        Construction| 4668|
|        Construction| 4665|
+--------------------+-----+
only showing top 10 rows



In [33]:
df_filtered=df.filter(df['value']>15000)

In [34]:
df_filtered.show(4)

+--------------------+--------+-----+--------------+---------+-----+
|         description|industry|level|          size|line_code|value|
+--------------------+--------+-----+--------------+---------+-----+
|Awareness of clim...|   total|    0|         total| C0300.01|18540|
|Awareness of clim...|   total|    0|6–19 employees| C0300.02|15810|
|Awareness of clim...|   total|    0|         total| C0300.02|21417|
|Awareness of gove...|   total|    0|6–19 employees| C0400.02|19209|
+--------------------+--------+-----+--------------+---------+-----+
only showing top 4 rows



In [35]:
df_group=df.groupby('industry').agg({'value':'avg'})

In [37]:
df_group.show()

+--------------------+------------------+
|            industry|        avg(value)|
+--------------------+------------------+
|Machinery & equip...|186.99303135888502|
|Arts & recreation...|100.21254355400697|
|           Insurance| 7.808362369337979|
|Education & training|210.86759581881532|
|Textile, clothing...|50.926829268292686|
|              Mining| 23.62369337979094|
|Other wholesale t...|445.25435540069685|
|          Publishing|29.822299651567945|
|  Commercial fishing| 10.19163763066202|
|             Finance| 39.90940766550523|
|Information media...|  83.1637630662021|
|Administrative & ...| 341.2369337979094|
|Computer systems ...|180.06271777003485|
|     Wholesale trade| 633.9721254355401|
|        Retail trade| 965.4355400696865|
|Agriculture, fore...| 194.1219512195122|
|Other machinery &...|48.930313588850176|
|        Construction|  1297.98606271777|
|      Other services| 267.9721254355401|
|Printing, publish...| 44.55052264808362|
+--------------------+------------

In [38]:
df.createOrReplaceTempView('df')

In [39]:
spark.sql('SELECT industry,value FROM df WHERE value < 500 and industry !="total"').show(10)

+--------------------+-----+
|            industry|value|
+--------------------+-----+
|  Commercial fishing|   27|
|  Forestry & logging|  126|
|              Mining|   72|
|Textile, clothing...|   96|
|Wood & paper product|  156|
|Printing, publish...|   72|
|Petroleum, coal, ...|  189|
|Non-metallic mine...|  108|
|       Metal product|  246|
|Transport and ind...|  285|
+--------------------+-----+
only showing top 10 rows

